In [102]:
# space_punc('" What a gorgeous man to work for')

' "  What a gorgeous man to work for'

In [101]:
import os
import pandas as pd
import re
# from collections import defaultdict

def space_punc(sent):
    new_sent=sent
    add=0
    for m in re.finditer(r'[\d\-\/\"\';,.!?\)\(]+',sent):
        if m.end(0)+add>=len(new_sent):
            return new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]
        else:
            new_sent=new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]+' '+new_sent[m.end(0)+add:]
            add+=2
    return new_sent

def replace_unk(words,test_w):
    try:
        pos=words.index(test_w.upper())
    except ValueError:
        try:
            pos=words.index(test_w.upper()+'S')
        except ValueError as e:
            try:
                pos=words.index(test_w.upper()+'ES')
            except ValueError as e:
                try:
                    pos=words.index(test_w.upper()[:-1]+'IES')
                except ValueError as e:
                    try:
                        pos=words.index(test_w.upper()+test_w.upper()[-1]+'ES')
                    except ValueError as e:
                        print (words,test_w,e)
    words[pos]='___'
    


if __name__=="__main__":
    trial2sent={}
    chimeras_data_dir='./eval_data/data-chimeras'
    
    data=pd.read_csv(os.path.join(chimeras_data_dir,'dataset.csv'))

    for index, row in data.iterrows():
        
        trial=str(row['TRIAL'])[:-2]
        if len(trial)<2:
            continue
        if '@@' not in str(row['PASSAGE']):
            continue
        
        if trial in trial2sent:
            continue
        
        test_ss=row['PASSAGE'].split('@@')
        test_ss_out=[]
        for test_s in test_ss:
            test_s.strip()
#             print (test_s)
            test_s=space_punc(test_s)
            words=test_s.split()
            replace_unk(words,row['NONCE'])
            test_ss_out.append(' '.join(words))
        
        trial2sent[trial]=(' @@ '.join(test_ss_out))

    
    for root, subdir, fnames in os.walk(chimeras_data_dir):
        for fn in fnames:
            if fn.endswith('fixed.test.txt'):
                infos=fn.split('.')
                sent_no=infos[1].upper()
                data_sub=pd.read_csv(os.path.join(chimeras_data_dir,fn),delimiter='\t',header=None)
                for line_num in range(len(data_sub)):
                    data_sub.at[line_num,1]=trial2sent[str(data_sub.at[line_num,0])+'_'+sent_no].strip()
                data_sub.to_csv(header=False,index=False,sep='\t',path_or_buf=os.path.join(chimeras_data_dir,fn+'.punct'))
                        
                        
    

It describes Aspergillas flavus which is a fungus mainly found in drought stressed DIVIRTH in the South-Eastern United States.  
 It would probably be more fun than listening to some self-deceiving gilgul spin her miserable DIVIRTH. 
It describes Aspergillas flavus which is a fungus mainly found in drought stressed CANBIM in the South-Eastern United States.  
 It would probably be more fun than listening to some self-deceiving gilgul spin her miserable CANBIM.  
 We 're getting food like CANBIM beef milk and flour and dividing into packages and taking into inaccessible places.  
 If you go where they sell CANBIM in the market they have it there now. 
It describes Aspergillas flavus which is a fungus mainly found in drought stressed THEND in the South-Eastern United States.  
 It would probably be more fun than listening to some self-deceiving gilgul spin her miserable THEND.  
 We 're getting food like THEND beef milk and flour and dividing into packages and taking into inaccessible pl

 It was feared that the SALBIA had been planted by terrorists and residents of Fairfield, Liverpool were evacuated.  
 The impact of the media on the conduct of air and SALBIA defense operations is substantially greater today than in any previous era.  
 But the chap in the office says a SALBIA went down there in the war and there was a landfall.  
 In particular, US naval and air superiority was paralleled by Soviet superiority in land-based SALBIA systems. 
Reeves'VELLUP playing has changed somewhat since Tin Machine 's debut three years ago.  
 Unfortunately no VELLUP or clavichord survives dated during the short period when Handel was in Hamburg. 
Reeves'CHUIKA playing has changed somewhat since Tin Machine 's debut three years ago.  
 Unfortunately no CHUIKA or clavichord survives dated during the short period when Handel was in Hamburg.  
 Before we provoke any lawsuits, let 's go back to what turned Rich on to playing CHUIKA in the first place.  
 Feature for feature, the Bate C

 A trip to Hun Stanton Cliffs was planned for fulmars, and to Holkham for BROONS.  
 Designed by Dave Thomas, close to the sea, surrounded by pine trees, and home BROONS.  
 On that occasion their predicament had been spotted and a boat had been sent out from Sharpness on a wild BROON chase. 
On the WHIZZES don't live long in the wild because of those enormous tails make them easy to catch for predators.  
 A trip to Hun Stanton Cliffs was planned for fulmars, and to Holkham for WHIZZES.  
 Designed by Dave Thomas, close to the sea, surrounded by pine trees, and home WHIZZES.  
 On that occasion their predicament had been spotted and a boat had been sent out from Sharpness on a wild WHIZZ chase.  
 Tao Chu, in contrast, seemed younger than his eight years and wore his heart embroidered like a WHIZZ on his sleeve.  
 A blue-grey heron glided to rest on a pebbly strand, and a cormorant flew high overhead like a WHIZZ. 
The DREAL, therefore, should keep its tail down, just as what a teach

 A man and his wife positioned themselves in the middle of one of the tree-lined avenues and sang ballads to a THRAND.  
 THRANDS are something of an obsession with me, especially when a player is trying to do something unusual with the instrument.  
 The title track has nice THRAND through it and a good Bowie vocal melody, although again, an unmemorable musical backing track. 
Well, in Brixton they might, but only Lloyd would have had a tenor BEMSTOM painted in gold on its bonnet.  
 Still, the BEMSTOM is cool, even if the vocals sound a little strained to me, lyrics too, actually. 
Well, in Brixton they might, but only Lloyd would have had a tenor ATRAD painted in gold on its bonnet.  
 Still, the ATRAD is cool, even if the vocals sound a little strained to me, lyrics too, actually.  
 We have three orchestras, several choirs, bands, an ATRAD quartet and a pipe band.  
 In the entry under his name, he wrote,"I have learned to play the ATRAD through my nose. 
Well, in Brixton they mig

 The BIBOT make-up is typical of the time with air-conditioned Mk2 stock supported by Mk1 braking coaches.  
 Shocking Hill!"was the 52 BIBOT conductor 's cabaret turn on his every approach to Notting Hill. 
At about the same age I was in a cafe and I spilt some pop on my GENKES.  
 It 's also the first International home GENKE in history that is reversible as well. 
At about the same age I was in a cafe and I spilt some pop on my STIENS.  
 It 's also the first International home STIEN in history that is reversible as well.  
 Such as how if you have sufficiently-stretchy pants you could take them off without first removing your STIENS.  
 It was in a cinema on 86th Street where he sat in the balcony, wearing an old sports jacket and open-necked STIEN. 
At about the same age I was in a cafe and I spilt some pop on my SCANKENS.  
 It 's also the first International home SCANKEN in history that is reversible as well.  
 Such as how if you have sufficiently-stretchy pants you could take 

The classical stuff I wrote is hard to do without the other instruments, you know ENEGES and stuff.  
 This is to include a selection of music in varying time signatures from the Highland ENEGE repertoire. 
The classical stuff I wrote is hard to do without the other instruments, you know BLERVES and stuff.  
 This is to include a selection of music in varying time signatures from the Highland BLERVE repertoire.  
 At the funeral, Cameron 's stepmother played the BLERVE while he stood with his arm around Alexia, feeling drained and empty.  
 Kids were treated to the sights and sounds of BLERVE marching bands, horse-drawn floats and siren-wailing antique fire trucks. 
The classical stuff I wrote is hard to do without the other instruments, you know CREERS and stuff.  
 This is to include a selection of music in varying time signatures from the Highland CREER repertoire.  
 At the funeral, Cameron 's stepmother played the CREER while he stood with his arm around Alexia, feeling drained an

'What a gorgeous man to work for , Anna confided , as she and Merrill stacked the ___ later . @@ If your ___ is not a continuous or self - cleaning model , you are in for a little more work .'